<a href="https://colab.research.google.com/github/GenAIUnplugged/langGraph_series/blob/main/langgraph_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install langgraph langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [3]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

1. Create an agent using create_react_agent

In [7]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="gpt-4o-mini",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='93a1ed4e-24b7-48ca-b844-d21aec5cc633'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_u1Ws5TqbcrvJSgCzIObKEvF8', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 56, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BW1Zjql2vvhKipwqdry2dAdaZIYNt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--20b07a68-6d62-4932-92b7-bf188bc734bb-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 

2. Configure an LLM using init_chat_model

In [8]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

model = init_chat_model(
    "gpt-4o-mini",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
)

3. Add a custom prompt : Static: A string is interpreted as a system message.

In [9]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='363ff17c-47bf-454a-b13a-d076f543938d'),
  AIMessage(content="I'm unable to provide information about the weather. However, you can check a reliable weather website or app for the latest updates.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 57, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BW1cuHQBKXqb2LEs9yG1WYWgEppt6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6decdd5e-b8bf-42fb-9843-02bf8cfc4902-0', usage_metadata={'input_tokens': 57, 'output_tokens': 27, 'total_tokens': 84, 'input_token_details':

4. Dynamic: A list of messages generated at runtime, based on input or configuration.

In [11]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='e660d83d-739c-4846-9a8f-f86b4af2dc47'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Eukx5efjqs85B2kklYrwaiA3', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 63, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BW1ds7gPrUQk5i6yhxFIOOHDSw2CO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f15afe7d-a7a3-4d5a-a36b-be5d635b7370-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 

5. Add memory \
To allow multi-turn conversations with an agent, you need to enable persistence by providing a checkpointer when creating an agent. At runtime, you need to provide a config containing thread_id — a unique identifier for the conversation (session):

In [12]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

6. Configure Structured Output \
To produce structured responses conforming to a schema, use the response_format parameter. The schema can be defined with a Pydantic model or TypedDict. \
Structured output requires an additional call to the LLM to format the response according to the schema.

In [13]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    response_format=WeatherResponse
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]

WeatherResponse(conditions="It's always sunny in San Francisco!")